In [1]:
import numpy as np
import pandas as pd

from sklearn.model_selection import KFold, StratifiedKFold, GroupKFold
from sklearn.metrics import roc_auc_score, mean_squared_error, accuracy_score

from tqdm import tqdm

import warnings; warnings.filterwarnings('ignore')

import lightgbm as lgb
import xgboost as xgb
import catboost as cb

from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
from xgboost import XGBClassifier
from sklearn.metrics import roc_auc_score


import matplotlib.pyplot as plt
import seaborn as sns

import time

In [2]:
import random
import os

def seed_everything(seed: int = 42):
    random.seed(seed)
    np.random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)

In [3]:
from pathlib import Path
DATA_PATH = Path("C:/ML_Projects/kaggle-workflow/data/")
OUTPUT_PATH = Path("C:/ML_Projects/kaggle-workflow/output/")

# General settings
SEED = 42
N_FOLDS = 5

# Metric choice placeholder
#Adjust depending on the competition
METRIC = "auc"

In [4]:
#Load Data
df = pd.read_csv(DATA_PATH / "Diabetes Prediction Challenge" / "train.csv")

In [5]:
df.columns

Index(['id', 'age', 'alcohol_consumption_per_week',
       'physical_activity_minutes_per_week', 'diet_score',
       'sleep_hours_per_day', 'screen_time_hours_per_day', 'bmi',
       'waist_to_hip_ratio', 'systolic_bp', 'diastolic_bp', 'heart_rate',
       'cholesterol_total', 'hdl_cholesterol', 'ldl_cholesterol',
       'triglycerides', 'gender', 'ethnicity', 'education_level',
       'income_level', 'smoking_status', 'employment_status',
       'family_history_diabetes', 'hypertension_history',
       'cardiovascular_history', 'diagnosed_diabetes'],
      dtype='object')

In [6]:
df = df.rename(columns = ({"alcohol_consumption_per_week":"alc",'physical_activity_minutes_per_week':"activity", 'diet_score':"diet",
                          'sleep_hours_per_day': "sleep", 'screen_time_hours_per_day':"screen", 'education_level':"edu",
                           'income_level':"inc", 'smoking_status': "smoke", 'employment_status':"empl", 'family_history_diabetes': "fam_his",
                          'hypertension_history': "hyp_his", 'cardiovascular_history': "card_his", 'diagnosed_diabetes': "label"} ))

In [7]:
# === Basic EDA ===

# Shape
print("Shape:", df.shape)

# Preview
display(df.head())

# Info
print("\nInfo:")
df.info()

# Target distribution 
print("\nTarget distribution:")
print(df["label"].value_counts(normalize=True))

# Numerical stats
print("\nNumerical summary:")
display(df.describe())

# Categorical stats (top categories)
print("\nCategorical summary:")
for col in df.select_dtypes(include=["object", "category"]).columns:
    print(f"{col}: {df[col].nunique()} unique")


Shape: (700000, 26)


,id,age,alc,activity,diet,sleep,screen,bmi,waist_to_hip_ratio,systolic_bp,...,gender,ethnicity,edu,inc,smoke,empl,fam_his,hyp_his,card_his,label
0,0,31,1,45,7.7,6.8,6.1,33.4,0.93,112,...,Female,Hispanic,Highschool,Lower-Middle,Current,Employed,0,0,0,1.0
1,1,50,2,73,5.7,6.5,5.8,23.8,0.83,120,...,Female,White,Highschool,Upper-Middle,Never,Employed,0,0,0,1.0
2,2,32,3,158,8.5,7.4,9.1,24.1,0.83,95,...,Male,Hispanic,Highschool,Lower-Middle,Never,Retired,0,0,0,0.0
3,3,54,3,77,4.6,7.0,9.2,26.6,0.83,121,...,Female,White,Highschool,Lower-Middle,Current,Employed,0,1,0,1.0
4,4,54,1,55,5.7,6.2,5.1,28.8,0.90,108,...,Male,White,Highschool,Upper-Middle,Never,Retired,0,1,0,1.0



Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 700000 entries, 0 to 699999
Data columns (total 26 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   id                  700000 non-null  int64  
 1   age                 700000 non-null  int64  
 2   alc                 700000 non-null  int64  
 3   activity            700000 non-null  int64  
 4   diet                700000 non-null  float64
 5   sleep               700000 non-null  float64
 6   screen              700000 non-null  float64
 7   bmi                 700000 non-null  float64
 8   waist_to_hip_ratio  700000 non-null  float64
 9   systolic_bp         700000 non-null  int64  
 10  diastolic_bp        700000 non-null  int64  
 11  heart_rate          700000 non-null  int64  
 12  cholesterol_total   700000 non-null  int64  
 13  hdl_cholesterol     700000 non-null  int64  
 14  ldl_cholesterol     700000 non-null  int64  
 15  triglycerides       700000 

,id,age,alc,activity,diet,sleep,screen,bmi,waist_to_hip_ratio,systolic_bp,diastolic_bp,heart_rate,cholesterol_total,hdl_cholesterol,ldl_cholesterol,triglycerides,fam_his,hyp_his,card_his,label
count,700000.000000,700000.000000,700000.000000,700000.000000,700000.000000,700000.000000,700000.000000,700000.000000,700000.000000,700000.000000,700000.000000,700000.000000,700000.000000,700000.000000,700000.000000,700000.000000,700000.000000,700000.000000,700000.000000,700000.000000
mean,349999.500000,50.359734,2.072411,80.230803,5.963695,7.002200,6.012733,25.874684,0.858766,116.294193,75.440924,70.167749,186.818801,53.823214,102.905854,123.081850,0.149401,0.181990,0.030324,0.623296
std,202072.738554,11.655520,1.048189,51.195071,1.463336,0.901907,2.022707,2.860705,0.037980,11.010390,6.825775,6.938722,16.730832,8.266545,19.022416,24.739397,0.356484,0.385837,0.171478,0.484560
min,0.000000,19.000000,1.000000,1.000000,0.100000,3.100000,0.600000,15.100000,0.680000,91.000000,51.000000,42.000000,117.000000,21.000000,51.000000,31.000000,0.000000,0.000000,0.000000,0.000000
25%,174999.750000,42.000000,1.000000,49.000000,5.000000,6.400000,4.600000,23.900000,0.830000,108.000000,71.000000,65.000000,175.000000,48.000000,89.000000,106.000000,0.000000,0.000000,0.000000,0.000000
50%,349999.500000,50.000000,2.000000,71.000000,6.000000,7.000000,6.000000,25.900000,0.860000,116.000000,75.000000,70.000000,187.000000,54.000000,103.000000,123.000000,0.000000,0.000000,0.000000,1.000000
75%,524999.250000,58.000000,3.000000,96.000000,7.000000,7.600000,7.400000,27.800000,0.880000,124.000000,80.000000,75.000000,199.000000,59.000000,116.000000,139.000000,0.000000,0.000000,0.000000,1.000000
max,699999.000000,89.000000,9.000000,747.000000,9.900000,9.900000,16.500000,38.400000,1.050000,163.000000,104.000000,101.000000,289.000000,90.000000,205.000000,290.000000,1.000000,1.000000,1.000000,1.000000



Categorical summary:
gender: 3 unique
ethnicity: 5 unique
edu: 4 unique
inc: 5 unique
smoke: 3 unique
empl: 4 unique


In [8]:
df.columns

Index(['id', 'age', 'alc', 'activity', 'diet', 'sleep', 'screen', 'bmi',
       'waist_to_hip_ratio', 'systolic_bp', 'diastolic_bp', 'heart_rate',
       'cholesterol_total', 'hdl_cholesterol', 'ldl_cholesterol',
       'triglycerides', 'gender', 'ethnicity', 'edu', 'inc', 'smoke', 'empl',
       'fam_his', 'hyp_his', 'card_his', 'label'],
      dtype='object')

In [9]:
target_col = ['label']
use_cols = ['age', 'alc', 'activity', 'diet', 'sleep', 'screen', 'bmi',
       'waist_to_hip_ratio', 'systolic_bp', 'diastolic_bp', 'heart_rate',
       'cholesterol_total', 'hdl_cholesterol', 'ldl_cholesterol',
       'triglycerides', 'gender', 'ethnicity', 'edu', 'inc', 'smoke', 'empl',
       'fam_his', 'hyp_his', 'card_his',]
X = df[use_cols].copy()
y=df[target_col].astype(int).copy()

In [10]:
cat_features =['gender', 'ethnicity', 'edu', 'inc', 'smoke', 'empl']

num_features = X.select_dtypes(include=["int64","float64"]).columns

In [11]:
test_cols = X.columns.tolist()
test_cols

['age',
 'alc',
 'activity',
 'diet',
 'sleep',
 'screen',
 'bmi',
 'waist_to_hip_ratio',
 'systolic_bp',
 'diastolic_bp',
 'heart_rate',
 'cholesterol_total',
 'hdl_cholesterol',
 'ldl_cholesterol',
 'triglycerides',
 'gender',
 'ethnicity',
 'edu',
 'inc',
 'smoke',
 'empl',
 'fam_his',
 'hyp_his',
 'card_his']

In [12]:
model = CatBoostClassifier(
        iterations=1000,
        learning_rate=0.05,
        depth=6,
        loss_function="Logloss",
        eval_metric="AUC",
        random_seed=42,
        verbose=False
    )

In [13]:
%%time
kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
oof = np.zeros(len(X))

for fold, (train_idx, val_idx) in enumerate(kf.split(X, y)):
    X_tr, X_va = X.iloc[train_idx], X.iloc[val_idx]
    y_tr, y_va = y.iloc[train_idx], y.iloc[val_idx]

    model = CatBoostClassifier(
        iterations=1000,
        learning_rate=0.05,
        depth=6,
        loss_function="Logloss",
        eval_metric="AUC",
        random_seed=42,
        verbose=False
    )

    model.fit(
        X_tr,
        y_tr,
        cat_features=cat_features,
        eval_set=(X_va, y_va),
        use_best_model=True
    )

    oof[val_idx] = model.predict_proba(X_va)[:, 1]

cv_auc = roc_auc_score(y, oof)
print("CV AUC:", cv_auc)

CV AUC: 0.7249441075628383
CPU times: total: 5h 23min 16s
Wall time: 39min 34s


In [14]:
from sklearn.base import clone
final_model = clone(model)
final_model.fit(X,y, cat_features = cat_features)

In [15]:
df_test = pd.read_csv(DATA_PATH / "Diabetes Prediction Challenge" / "test.csv")

In [16]:
df_test = df_test.rename(columns = ({"alcohol_consumption_per_week":"alc",'physical_activity_minutes_per_week':"activity", 'diet_score':"diet",
                          'sleep_hours_per_day': "sleep", 'screen_time_hours_per_day':"screen", 'education_level':"edu",
                           'income_level':"inc", 'smoking_status': "smoke", 'employment_status':"empl", 'family_history_diabetes': "fam_his",
                          'hypertension_history': "hyp_his", 'cardiovascular_history': "card_his", 'diagnosed_diabetes': "label"} ))

In [17]:
# Shape
print("Shape:", df_test.shape)

# Preview
display(df_test.head())

# Info
print("\nInfo:")
df_test.info()

# Numerical stats
print("\nNumerical summary:")
display(df_test.describe())


Shape: (300000, 25)


,id,age,alc,activity,diet,sleep,screen,bmi,waist_to_hip_ratio,systolic_bp,...,triglycerides,gender,ethnicity,edu,inc,smoke,empl,fam_his,hyp_his,card_his
0,700000,45,4,100,4.3,6.8,6.2,25.5,0.84,123,...,111,Female,White,Highschool,Middle,Former,Employed,0,0,0
1,700001,35,1,87,3.5,4.6,9.0,28.6,0.88,120,...,145,Female,White,Highschool,Middle,Never,Unemployed,0,0,0
2,700002,45,1,61,7.6,6.8,7.0,28.5,0.94,112,...,184,Male,White,Highschool,Low,Never,Employed,0,0,0
3,700003,55,2,81,7.3,7.3,5.0,26.9,0.91,114,...,128,Male,White,Graduate,Middle,Former,Employed,0,0,0
4,700004,77,2,29,7.3,7.6,8.5,22.0,0.83,131,...,133,Male,White,Graduate,Low,Current,Unemployed,0,0,0



Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 300000 entries, 0 to 299999
Data columns (total 25 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   id                  300000 non-null  int64  
 1   age                 300000 non-null  int64  
 2   alc                 300000 non-null  int64  
 3   activity            300000 non-null  int64  
 4   diet                300000 non-null  float64
 5   sleep               300000 non-null  float64
 6   screen              300000 non-null  float64
 7   bmi                 300000 non-null  float64
 8   waist_to_hip_ratio  300000 non-null  float64
 9   systolic_bp         300000 non-null  int64  
 10  diastolic_bp        300000 non-null  int64  
 11  heart_rate          300000 non-null  int64  
 12  cholesterol_total   300000 non-null  int64  
 13  hdl_cholesterol     300000 non-null  int64  
 14  ldl_cholesterol     300000 non-null  int64  
 15  triglycerides       300000 

,id,age,alc,activity,diet,sleep,screen,bmi,waist_to_hip_ratio,systolic_bp,diastolic_bp,heart_rate,cholesterol_total,hdl_cholesterol,ldl_cholesterol,triglycerides,fam_his,hyp_his,card_his
count,300000.000000,300000.000000,300000.000000,300000.000000,300000.000000,300000.000000,300000.000000,300000.000000,300000.000000,300000.000000,300000.000000,300000.000000,300000.000000,300000.000000,300000.000000,300000.000000,300000.000000,300000.000000,300000.000000
mean,849999.500000,50.432397,2.089693,92.349087,5.945838,6.997795,6.011278,25.881906,0.859007,116.374117,75.396013,70.048350,187.308620,53.813557,103.416083,123.538480,0.152920,0.184410,0.033110
std,86602.684716,11.938741,1.066214,62.187399,1.481068,0.914693,2.060472,2.894289,0.038523,11.252146,6.950340,7.090543,18.413053,8.398126,20.571855,28.965441,0.359911,0.387819,0.178924
min,700000.000000,19.000000,1.000000,1.000000,0.100000,3.100000,0.600000,15.100000,0.690000,91.000000,51.000000,42.000000,107.000000,22.000000,51.000000,31.000000,0.000000,0.000000,0.000000
25%,774999.750000,42.000000,1.000000,51.000000,5.000000,6.400000,4.600000,23.900000,0.830000,108.000000,71.000000,65.000000,174.000000,48.000000,89.000000,104.000000,0.000000,0.000000,0.000000
50%,849999.500000,50.000000,2.000000,77.000000,6.000000,7.000000,6.000000,25.900000,0.860000,116.000000,75.000000,70.000000,187.000000,54.000000,103.000000,123.000000,0.000000,0.000000,0.000000
75%,924999.250000,59.000000,3.000000,115.000000,7.000000,7.600000,7.400000,27.800000,0.890000,124.000000,80.000000,75.000000,200.000000,60.000000,117.000000,142.000000,0.000000,0.000000,0.000000
max,999999.000000,89.000000,9.000000,748.000000,9.900000,9.900000,15.900000,38.300000,1.050000,170.000000,104.000000,101.000000,285.000000,91.000000,226.000000,290.000000,1.000000,1.000000,1.000000


In [18]:
X_test = df_test[test_cols]

In [19]:
test_preds = final_model.predict_proba(X_test)[:,1]
test_preds

array([0.50841356, 0.67257261, 0.78609538, ..., 0.55052084, 0.6154804 ,
       0.60055301], shape=(300000,))

In [20]:
#sub.to_csv("dia_submission_cat_1.csv", index=False)

In [21]:
### LB = 0.69708, CV = 0.7249441075628383

In [22]:
ENS_PATH = Path("C:\ML_Projects\kaggle-workflow\competitions\Diabetes Prediction Challenge\Ens")
test_pred_models = pd.read_csv(ENS_PATH / "test_pred_models.csv")
test_pred_models.head()

,id,prediction_MLP,prediction_LGBM
0,0,0.602803,0.505413
1,1,0.642024,0.675915
2,2,0.685453,0.765060
3,3,0.633595,0.408179
4,4,0.867566,0.904486


In [23]:
test_pred_models["prediction_Cat"] = test_preds
test_pred_models.head()

,id,prediction_MLP,prediction_LGBM,prediction_Cat
0,0,0.602803,0.505413,0.508414
1,1,0.642024,0.675915,0.672573
2,2,0.685453,0.765060,0.786095
3,3,0.633595,0.408179,0.423093
4,4,0.867566,0.904486,0.921526


In [24]:
test_pred_models.to_csv("test_pred_models.csv", index=False)